# CUPS Report Analisis Function

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
df = pd.read_excel('/home/raulserrano/DATA_SCIENCE/proyectos/CUPX/cups-examples/InformeCUPS_AMOROS.xlsx', sheet_name=1)
df.drop('Tipo lectura', axis=1, inplace=True)
df.head(40)

,Fecha lectura anterior,Fecha lectura,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6
0,30/11/2020,01/12/2020,0,1122,771,0,0,1455
1,01/12/2020,01/01/2021,33773,23115,0,0,0,50226
2,01/01/2021,01/02/2021,36772,24989,0,0,0,44394
3,01/02/2021,01/03/2021,32760,22241,0,0,0,40610
4,01/03/2021,01/04/2021,0,33989,23429,0,0,50449
5,01/04/2021,01/05/2021,0,0,0,36068,25461,49314
6,01/05/2021,31/05/2021,0,0,0,33233,24018,47816
7,31/05/2021,30/06/2021,0,0,30461,22779,0,37340
8,30/06/2021,31/07/2021,32269,23738,0,0,0,44815
9,31/07/2021,31/08/2021,0,0,37129,26828,0,52685


In [3]:
def solo_dates(df):
    df_dates = df[['Fecha lectura anterior', 'Fecha lectura']].copy()

    # Eliminar espacios en blanco antes de la conversión
    df_dates['Fecha lectura anterior'] = df_dates['Fecha lectura anterior'].str.strip()
    df_dates['Fecha lectura'] = df_dates['Fecha lectura'].str.strip()

    # Convertir las columnas a datetime y manejar errores
    df_dates['Fecha lectura anterior'] = pd.to_datetime(df_dates['Fecha lectura anterior'], format='%d/%m/%Y')
    df_dates['Fecha lectura'] = pd.to_datetime(df_dates['Fecha lectura'], format='%d/%m/%Y')

    # Verificar si hay valores NaT que no se pudieron convertir
    # print(df_dates.dtypes)  # Verifica los tipos de datos
    
    return df_dates

In [4]:
dates_df = solo_dates(df)
dates_df.head(10)

,Fecha lectura anterior,Fecha lectura
0,2020-11-30,2020-12-01
1,2020-12-01,2021-01-01
2,2021-01-01,2021-02-01
3,2021-02-01,2021-03-01
4,2021-03-01,2021-04-01
5,2021-04-01,2021-05-01
6,2021-05-01,2021-05-31
7,2021-05-31,2021-06-30
8,2021-06-30,2021-07-31
9,2021-07-31,2021-08-31


In [5]:
def dates_p6(df):
    df_dates = df[['Fecha lectura anterior', 'Fecha lectura', 'Consumo P6']].copy()

    # Eliminar espacios en blanco antes de la conversión
    df_dates['Fecha lectura anterior'] = df_dates['Fecha lectura anterior'].str.strip()
    df_dates['Fecha lectura'] = df_dates['Fecha lectura'].str.strip()

    # Convertir las columnas a datetime y manejar errores
    df_dates['Fecha lectura anterior'] = pd.to_datetime(df_dates['Fecha lectura anterior'], format='%d/%m/%Y')
    df_dates['Fecha lectura'] = pd.to_datetime(df_dates['Fecha lectura'], format='%d/%m/%Y')

    # Añadir la columna de "Duración días" calculando los días entre las dos fechas
    df_dates['Duración días'] = (df_dates['Fecha lectura'] - df_dates['Fecha lectura anterior']).dt.days
    
    # Verificar si hay valores NaT que no se pudieron convertir
    # print(df_dates.dtypes)  # Verifica los tipos de datos
    
    return df_dates

In [6]:
df_dates_p6 = dates_p6(df)
df_dates_p6

,Fecha lectura anterior,Fecha lectura,Consumo P6,Duración días
0,2020-11-30,2020-12-01,1455,1
1,2020-12-01,2021-01-01,50226,31
2,2021-01-01,2021-02-01,44394,31
3,2021-02-01,2021-03-01,40610,28
4,2021-03-01,2021-04-01,50449,31
5,2021-04-01,2021-05-01,49314,30
6,2021-05-01,2021-05-31,47816,30
7,2021-05-31,2021-06-30,37340,30
8,2021-06-30,2021-07-31,44815,31
9,2021-07-31,2021-08-31,52685,31


#### Fx to Eliminate 1 day intervals

In [7]:
def eliminar_filas_un_dia(df):
    # Crear una nueva copia del DataFrame sin las filas que tengan un solo día
    df_sin_filas_un_dia = df[df['Fecha lectura anterior'] != df['Fecha lectura']].copy().reset_index().drop(columns=['index'])
    
    return df_sin_filas_un_dia

#### Fx to divide rows in different months

In [8]:
def dividir_filas(df):
    filas_divididas = []
    dates_df = dates_p6(df)

    # Añadir la columna de "Duración días" calculando los días entre las dos fechas
    dates_df['Duración días'] = (dates_df['Fecha lectura'] - dates_df['Fecha lectura anterior']).dt.days
    
    for _, row in dates_df.iterrows():
        start_date = row['Fecha lectura anterior']
        end_date = row['Fecha lectura']
        consum = row['Consumo P6']
        total_days = row['Duración días']  # total_days es ahora un número entero

        # Si las fechas están en meses diferentes, dividir la fila
        if start_date.month != end_date.month or start_date.year != end_date.year:
            # Primera parte: desde start_date hasta el último día del mes de start_date
            last_day_of_start_month = pd.Timestamp(start_date.year, start_date.month, 
                                                   pd.Timestamp(start_date.year, start_date.month, 1).days_in_month)
            days_part1 = (last_day_of_start_month - start_date).days + 1  # Días en la primera parte
            
            prop_days1 = days_part1 / total_days
            prop_cons1 = prop_days1 * consum

            filas_divididas.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': last_day_of_start_month,
                'Duración días': days_part1,
                'Proporción días': prop_days1,  # Proporción de la primera parte
                'Consumo P6': round(prop_cons1, 0)
            })

            # Segunda parte: desde el primer día del siguiente mes hasta end_date
            first_day_of_next_month = last_day_of_start_month + timedelta(days=1)
            days_part2 = (end_date - first_day_of_next_month).days + 1  # Días en la segunda parte
            
            prop_days2 = days_part2 / total_days
            prop_cons2 = round(prop_days2 * consum, 0)

            filas_divididas.append({
                'Fecha lectura anterior': first_day_of_next_month, 
                'Fecha lectura': end_date,
                'Duración días': days_part2,
                'Proporción días': prop_days2,  # Proporción de la segunda parte
                'Consumo P6': round(consum - prop_cons1, 0)
            })
        else:
            # Si no abarcan más de un mes, se queda la fila tal cual con su duración completa
            filas_divididas.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': end_date,
                'Duración días': total_days,
                'Proporción días': 1,  # Proporción completa
                'Consumo P6':consum
            })
        
        # Crear DataFrame con las filas divididas y eliminar las filas de un solo día
    split_intervals = pd.DataFrame(filas_divididas)
    final_df = eliminar_filas_un_dia(split_intervals)

    return final_df, filas_divididas

# Aplicamos la función para dividir las filas y mostrar el resultado
df_dividido, filas_dic = dividir_filas(df)

# Mostrar el DataFrame resultante
df_dividido['Consumo OG'] = df.loc[:,'Consumo P6']
df_dividido.head(50)

,Fecha lectura anterior,Fecha lectura,Duración días,Proporción días,Consumo P6,Consumo OG
0,2020-12-01,2020-12-31,31,1.000000,50226.0,1455
1,2021-01-01,2021-01-31,31,1.000000,44394.0,50226
2,2021-02-01,2021-02-28,28,1.000000,40610.0,44394
3,2021-03-01,2021-03-31,31,1.000000,50449.0,40610
4,2021-04-01,2021-04-30,30,1.000000,49314.0,50449
5,2021-05-01,2021-05-31,30,1.000000,47816.0,49314
6,2021-06-01,2021-06-30,30,1.000000,36095.0,47816
7,2021-07-01,2021-07-31,31,1.000000,43369.0,37340
8,2021-08-01,2021-08-31,31,1.000000,50985.0,44815
9,2021-09-01,2021-09-30,30,1.000000,44717.0,52685


In [14]:
def dividir_filas(df):
    filas_divididas = []
    dates_df = dates_p6(df)
    
    for _, row in dates_df.iterrows():
        start_date = row['Fecha lectura anterior']
        end_date = row['Fecha lectura']
        consum = row['Consumo P6']  # Añadimos el consumo que vamos a distribuir
        total_days = row['Duración días']  # Número total de días en el intervalo

        # Si las fechas están en meses diferentes, dividir la fila
        if start_date.month != end_date.month or start_date.year != end_date.year:
            # Primera parte: desde start_date hasta el último día del mes de start_date
            last_day_of_start_month = pd.Timestamp(start_date.year, start_date.month, 
                                                   pd.Timestamp(start_date.year, start_date.month, 1).days_in_month)
            days_part1 = (last_day_of_start_month - start_date).days + 1  # Días en la primera parte
            prop_days1 = days_part1 / total_days  # Proporción de días para la primera parte
            consum_part1 = round(prop_days1 * consum, 0)  # Ajustamos el consumo de la primera parte

            # Segunda parte: desde el primer día del siguiente mes hasta end_date
            first_day_of_next_month = last_day_of_start_month + timedelta(days=1)
            days_part2 = (end_date - first_day_of_next_month).days + 1  # Días en la segunda parte
            prop_days2 = days_part2 / total_days  # Proporción de días para la segunda parte
            consum_part2 = consum - consum_part1  # Ajustamos la segunda parte para asegurar que la suma sea igual al consumo total

            filas_divididas.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': last_day_of_start_month,
                'Duración días': days_part1,
                'Proporción días': prop_days1,
                'Consumo P6': consum_part1  # Consumo ajustado para la primera parte
            })

            filas_divididas.append({
                'Fecha lectura anterior': first_day_of_next_month, 
                'Fecha lectura': end_date,
                'Duración días': days_part2,
                'Proporción días': prop_days2,
                'Consumo P6': consum_part2  # Consumo ajustado para la segunda parte
            })
        else:
            # Si no abarcan más de un mes, se queda la fila tal cual con su consumo total
            filas_divididas.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': end_date,
                'Duración días': total_days,
                'Proporción días': 1,  # Proporción completa
                'Consumo P6': consum  # Consumo total
            })
    
    # Crear DataFrame con las filas divididas después del bucle
    final_df = pd.DataFrame(filas_divididas)
    final_df = eliminar_filas_un_dia(final_df)

    return final_df, filas_divididas

# Aplicamos la función para dividir las filas y mostrar el resultado
df_dividido, filas_dic = dividir_filas(df)

# Mostrar el DataFrame resultante
df_dividido['Consumo OG'] = df.loc[:,'Consumo P6']
df_dividido.head(50)

,Fecha lectura anterior,Fecha lectura,Duración días,Proporción días,Consumo P6,Consumo OG
0,2020-12-01,2020-12-31,31,1.000000,50226.0,1455
1,2021-01-01,2021-01-31,31,1.000000,44394.0,50226
2,2021-02-01,2021-02-28,28,1.000000,40610.0,44394
3,2021-03-01,2021-03-31,31,1.000000,50449.0,40610
4,2021-04-01,2021-04-30,30,1.000000,49314.0,50449
5,2021-05-01,2021-05-31,30,1.000000,47816.0,49314
6,2021-06-01,2021-06-30,30,1.000000,36095.0,47816
7,2021-07-01,2021-07-31,31,1.000000,43369.0,37340
8,2021-08-01,2021-08-31,31,1.000000,50985.0,44815
9,2021-09-01,2021-09-30,30,1.000000,44717.0,52685


### Prueba tres, ajustando precisión

In [20]:
tot_OG = df_dividido.loc[:,'Consumo OG'].sum()
tot_mod = df_dividido.loc[:,'Consumo P6'].sum()
print(f'OG: {tot_OG}\nMod:{tot_mod}\nDiff = {tot_OG - tot_mod}')

OG: 1552331
Mod:1550876.0
Diff = 1455.0


### Prueba cuatro
-   Ajuste de precision redistribuyendo la fila 1 dia a la que sea de su mismo mes

In [24]:
def redistribuir_filas_un_dia(df):
    for i, row in df.iterrows():
        if row['Duración días'] == 1:
            # Extraemos el mes de la fila actual
            mes_actual = row['Fecha lectura'].month

            # Comprobamos la fila anterior
            if i > 0 and df.loc[i-1, 'Fecha lectura'].month == mes_actual:
                # Si la fila anterior es del mismo mes, sumamos el consumo a la anterior
                df.loc[i-1, 'Consumo P6'] += row['Consumo P6']
            # Si la fila anterior no es del mismo mes, comprobamos la siguiente fila
            elif i < len(df) - 1 and df.loc[i+1, 'Fecha lectura'].month == mes_actual:
                # Si la fila siguiente es del mismo mes, sumamos el consumo a la siguiente
                df.loc[i+1, 'Consumo P6'] += row['Consumo P6']
    
    # Eliminar filas de un solo día después de redistribuir el consumo
    df_sin_filas_un_dia = df[df['Duración días'] != 1].copy().reset_index(drop=True)
    return df_sin_filas_un_dia

def dividir_filas(df):
    filas_divididas = []
    dates_df = dates_p6(df)
    
    for _, row in dates_df.iterrows():
        start_date = row['Fecha lectura anterior']
        end_date = row['Fecha lectura']
        consum = row['Consumo P6']
        total_days = row['Duración días']

        # Si las fechas están en meses diferentes, dividir la fila
        if start_date.month != end_date.month or start_date.year != end_date.year:
            # Primera parte: desde start_date hasta el último día del mes de start_date
            last_day_of_start_month = pd.Timestamp(start_date.year, start_date.month, 
                                                   pd.Timestamp(start_date.year, start_date.month, 1).days_in_month)
            days_part1 = (last_day_of_start_month - start_date).days + 1
            prop_days1 = days_part1 / total_days
            consum_part1 = round(prop_days1 * consum, 0)

            filas_divididas.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': last_day_of_start_month,
                'Duración días': days_part1,
                'Proporción días': prop_days1,
                'Consumo P6': consum_part1
            })

            # Segunda parte: desde el primer día del siguiente mes hasta end_date
            first_day_of_next_month = last_day_of_start_month + timedelta(days=1)
            days_part2 = (end_date - first_day_of_next_month).days + 1
            prop_days2 = days_part2 / total_days
            consum_part2 = consum - consum_part1  # Asegurar que la suma sea igual al consumo original

            filas_divididas.append({
                'Fecha lectura anterior': first_day_of_next_month, 
                'Fecha lectura': end_date,
                'Duración días': days_part2,
                'Proporción días': prop_days2,
                'Consumo P6': consum_part2
            })
        else:
            filas_divididas.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': end_date,
                'Duración días': total_days,
                'Proporción días': 1,
                'Consumo P6': consum
            })
    
    # Crear DataFrame con las filas divididas y redistribuir el consumo de las filas con un solo día
    final_df = pd.DataFrame(filas_divididas)
    final_df = redistribuir_filas_un_dia(final_df)

    return final_df, filas_divididas

# Aplicamos la función para dividir las filas y mostrar el resultado
df_dividido, filas_dic = dividir_filas(df)

# Mostrar el DataFrame resultante
df_dividido['Consumo OG'] = df['Consumo P6']
df_dividido.head(50)

,Fecha lectura anterior,Fecha lectura,Duración días,Proporción días,Consumo P6,Consumo OG
0,2020-12-01,2020-12-31,31,1.000000,50226.0,1455
1,2021-01-01,2021-01-31,31,1.000000,44394.0,50226
2,2021-02-01,2021-02-28,28,1.000000,40610.0,44394
3,2021-03-01,2021-03-31,31,1.000000,50449.0,40610
4,2021-04-01,2021-04-30,30,1.000000,49314.0,50449
5,2021-05-01,2021-05-31,30,1.000000,49061.0,49314
6,2021-06-01,2021-06-30,30,1.000000,37541.0,47816
7,2021-07-01,2021-07-31,31,1.000000,45069.0,37340
8,2021-08-01,2021-08-31,31,1.000000,52527.0,44815
9,2021-09-01,2021-09-30,30,1.000000,46315.0,52685


In [25]:
tot_OG = df_dividido.loc[:,'Consumo OG'].sum()
tot_mod = df_dividido.loc[:,'Consumo P6'].sum()
print(f'OG: {tot_OG}\nMod:{tot_mod}\nDiff = {tot_OG - tot_mod}')

OG: 1552331
Mod:1550876.0
Diff = 1455.0


## CLASS DEFINITION

---

In [9]:
class CupsARP:

    def __init__(self, path):
        self.path = path
        self.page_list = []

        self.df_cups_list = pd.read_excel(path, sheet_name=0)
        self.df_activa = pd.read_excel(path, sheet_name=1).drop('Tipo lectura', axis=1)
        self.df_maxim = pd.read_excel(path, sheet_name=2)
        self.df_react = pd.read_excel(path, sheet_name=3)

        self.page_list.extend([self.df_cups_list, self.df_activa, self.df_maxim, self.df_react])

    
    def load_page(self, param):
        return self.page_list[param]


    def ly_consum(self):
        self.consums = {}
        
        self.tot_cons = self.df_activa[['Fecha lectura','Consumo P1', 'Consumo P2',
       'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].sum().sum()
        self.consums['TOTAL'] = self.tot_cons

        for i in range(1,7):
             name = f'P{i}'
             
             cons = self.df_activa.loc[:,f'Consumo P{i}'].sum()
             self.consums[name] = cons

        return self.consums
    
    def year_table(self):
        # Crear una tabla pivotada con los meses y periodos P1 a P6
        tabla = self.df_activa[['Fecha lectura', 'Consumo P1', 'Consumo P2',
                                'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].copy()

        # Convertir las fechas a meses (ej. Enero, Febrero, ...)
        tabla['Mes'] = pd.to_datetime(self.df_activa['Fecha lectura'], format="%d/%m/%Y").dt.strftime('%B').str.upper()

        # Agrupar los datos por 'Mes' y sumar los consumos para cada mes y cada periodo
        tabla_agrupada = tabla.groupby('Mes').sum()

        # Calcular el total por cada mes
        tabla_agrupada['Total mes'] = tabla_agrupada.sum(axis=1)

        # Calcular el total por cada periodo (columna)
        total_periodos = tabla_agrupada[['Consumo P1', 'Consumo P2', 'Consumo P3',
                                         'Consumo P4', 'Consumo P5', 'Consumo P6']].sum()

        # Añadir una fila de total de periodos al final de la tabla
        tabla_agrupada.loc['Total periodo'] = total_periodos
        tabla_agrupada.loc['Total periodo', 'Total mes'] = total_periodos.sum()

        return tabla_agrupada

